In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 100)
import missingno as msno
import re
from sklearn.preprocessing import LabelEncoder
import pickle
%matplotlib inline

In [2]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
train_df.head(6).transpose()

In [ ]:
print (train_df.info())

In [ ]:
#目前使用的特征
feature = ['loan_amnt',#借贷总额
           'funded_amnt',
           'funded_amnt_inv',
           'term', 
           'int_rate',
           'installment',
           'grade',
           'sub_grade',
           'emp_length',#从业时长
           'home_ownership',
           'annual_inc',#年收入,有4个缺失值
           'verification_status',#object
           'loan_status',#object
           'pymnt_plan',#object,是否已经为贷款实施了付款计划
           'dti',#月付与月收比率
           'mths_since_last_record',#有大量缺失值
           'pub_rec',#有缺失值
           'revol_bal',
           'revol_util',
           'total_acc',#信用额度
           'initial_list_status',#object
           'out_prncp',
           'out_prncp_inv',
           'total_pymnt',
           'total_pymnt_inv',
           'total_rec_prncp',
           'total_rec_int',
           'total_rec_late_fee',
           'recoveries']

In [3]:
#Combine into data:
train_df['source']= 'train'
test_df['source'] = 'test'
dataset=pd.concat([train_df,test_df],ignore_index=True)
dataset.shape

(887379, 65)

In [5]:
# Check missing:
print(dataset.isnull().sum())

acc_now_delinq                 177476
addr_state                          0
all_util                       866007
annual_inc                          4
annual_inc_joint               886868
application_type                    0
collection_recovery_fee             0
collections_12_mths_ex_med        145
desc                           761350
dti                                 0
dti_joint                      886870
earliest_cr_line                   29
emp_length                          0
emp_title                       51457
funded_amnt                         0
funded_amnt_inv                     0
grade                               0
home_ownership                      0
il_util                        868762
initial_list_status                 0
inq_fi                         866007
inq_last_12m                   866007
installment                         0
int_rate                            0
issue_d                             0
loan_amnt                           0
loan_status 

In [4]:
# XGboost is good at dealing with numbers but definitely not good when dealing with string. 
# So the solution is neither we drop them or transform them.
drop_elements = ['annual_inc_joint',# high missing rate
                 'dti_joint',# high missing rate
                 'verification_status_joint',# high missing rate
                 'issue_d',#The month which the loan was funded
                 'emp_title',
                 'desc',
                 'title',
                 'desc', 
                 'zip_code',
                 'addr_state',
                 'earliest_cr_line', #借款人最早报告的信用额度被打开的月份
                 'application_type']
dataset.drop(drop_elements,axis=1,inplace=True)

# Mapping emp_length
emp_length_mapping = {'< 1 year': 1, '1 year': 2, '2 years': 3, '3 years': 4, '4 years': 5, '5 years': 6, '6 years': 7, '7 years': 8, '8 years': 9, '9 years': 10, '10 years': 11, '10+ years': 12}
dataset['emp_length'] = dataset['emp_length'].map(emp_length_mapping)
dataset['emp_length'] = dataset['emp_length'].fillna(0)

# Mapping term: 36 months->36
dataset['term']=dataset['term'].apply(lambda row:int(re.findall(r"\d+\.?\d*",row)[0]))

# Mapping Category feature
category_feature=['grade','sub_grade','home_ownership','verification_status','pymnt_plan','loan_status','purpose','initial_list_status']
dataset[category_feature]=dataset[category_feature].apply(LabelEncoder().fit_transform)

#One-Hot Coding¶
# dataset = pd.get_dummies(dataset, columns=category_feature)
    
#Handle datetime，将Feb-1985转换成1985-02-01
#     dataset['earliest_cr_line'] =  pd.to_datetime(dataset['earliest_cr_line'], format='%b-%Y')
#     dataset['issue_d'] = pd.to_datetime(dataset['issue_d'], format='%b-%Y')

In [5]:
dataset.columns

Index(['acc_now_delinq', 'all_util', 'annual_inc', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'dti', 'emp_length', 'funded_amnt',
       'funded_amnt_inv', 'grade', 'home_ownership', 'il_util',
       'initial_list_status', 'inq_fi', 'inq_last_12m', 'installment',
       'int_rate', 'loan_amnt', 'loan_status', 'max_bal_bc', 'member_id',
       'mths_since_last_major_derog', 'mths_since_last_record',
       'mths_since_rcnt_il', 'open_acc_6m', 'open_il_12m', 'open_il_24m',
       'open_il_6m', 'open_rv_12m', 'open_rv_24m', 'out_prncp',
       'out_prncp_inv', 'policy_code', 'pub_rec', 'purpose', 'pymnt_plan',
       'recoveries', 'revol_bal', 'revol_util', 'source', 'sub_grade', 'term',
       'tot_coll_amt', 'tot_cur_bal', 'total_acc', 'total_bal_il',
       'total_cu_tl', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int',
       'total_rec_late_fee', 'total_rec_prncp', 'total_rev_hi_lim',
       'verification_status'],
      dtype='object')

In [ ]:
print (train_df.info())

In [ ]:
train_df.head().transpose()

In [6]:
train = dataset.loc[dataset['source']=='train']
test = dataset.loc[dataset['source']=='test']

In [7]:
train.drop('source',axis=1,inplace=True)
test.drop(['source','acc_now_delinq'],axis=1,inplace=True)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
train.to_csv('../dump/train_modified_label.csv',index=False)
test.to_csv('../dump/test_modified_label.csv',index=False)